# Numerical Methods for Manu Body Physics, Assignment #4

Yoav Zack, ID 211677398

In [ ]:
using LinearAlgebra
using Statistics
using Plots
using LsqFit
using Printf
using BenchmarkTools
theme(:dracula)

## Question 1: 2D XY model and the Kosterlitz-Thouless transition

We want to implement a Monte-Carlo algorithm using Wolff Clusters for the 2D XY Model. In order to do so, we need the following functions:
1. `WolffUpdate()` - Updates a grid of spins 
2. `selectCluster()` - Selects a cluster according to Wolff method
3. `nearestNeighboors()` - Returns a list of nearest neighboors of a given site assuming given boundary conditions
It will be easier for us to work if some of the data will be stores ina a `Grid` structure, as follows:

In [ ]:
struct Grid
    N::Integer
    grid::Matrix{Real}
    J::Real
    β::Real
    bc::String
    
    function Grid(N::Integer, J::Real, T::Real, bc::String)
        @assert N > 0 "Grid size must be positive integer"
        @assert J > 0 "Interaction coefficient J must be positive"
        @assert T > 0 "Temperature must be positive"

        if !(bc in ["periodic", "closed"])
            error("Boundacy conditions are either periodic or closed")
        end
        
        β = 1. / T
        grid = (2*π) .* rand(N,N)
        new(N, grid, J, β, bc)
    end
end

We will also define a simple function which plots the grid data ina a heatmap:

In [ ]:
function show(grid::Grid)
    h = heatmap(grid.grid)
    heatmap!(h, xlim=(1,grid.N), ylim=(1,grid.N), xticks=0, yticks=0)
    heatmap!(h, aspect_ratio=:equal, frame=:on)
    heatmap!(title=@sprintf("grid with β=%.2f", grid.β))
    return h
end

Now, we can start working on the functions mentioned above. First, the `nearestNeighboors()` function:

In [ ]:
function nearestNeighboors(i::Integer, j::Integer, grid::Grid)
    N = grid.N
    if grid.bc == "periodic"
        i = (i-1 % N) + 1
        j = (j-1 % N) + 1
    end

    @assert i >= 1 && j >= 1 && i <= N && j <= N "Invalid spin index"
    neighboors = [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]
    
    if grid.bc == "periodic"
        return [mod.(a .- 1, N) .+ 1 for a in neighboors]
    elseif grid.bc == "closed"
        return [(x,y) for (x,y) in neighboors if (x>=1) && (y>=1) && (x<=N) && (y<=N)]
    else
        error("Invalid boundary condition in nearestNeighboors()")
    end
end

Notice that we solved it for both periodic and closed boundary conditions. Next, we work on the Wolff part - selecting a cluster:

In [ ]:
function selectCluster(θe::Real, i::Integer, j::Integer, grid::Grid)
    cluster = [(i,j)]
    queue = nearestNeighboors(i, j, grid)

    θi = grid.grid[i,j]
    prodi = cos(θi - θe)
    while !isempty(queue)
        s = pop!(queue)
        if s in cluster
            continue
        end
        θj = grid.grid[s[1], s[2]]
        prodj = cos(θj - θe)
        p = 1 - exp(min(0, -2*grid.β*grid.J*prodi*prodj))
        if p > rand()
            append!(cluster, [s])
            append!(queue, nearestNeighboors(s[1], s[2], grid))
        end
    end

    return cluster
end

This gives us a list of indices to by sliced by. Next, we write a function which does a single Wolff spin flip according to the function given in the assignment:

In [ ]:
function iterationWolff!(grid::Grid)
    i = rand(1:grid.N)
    j = rand(1:grid.N)

    θe = 2*π*rand()
    cluster = selectCluster(θe, i, j, grid)
    
    for s in cluster
        grid.grid[s[1], s[2]] = mod(grid.grid[s[1], s[2]] - 2*θe, 2*π)
    end
end

Let's benchmark our function to see if it is any good:

In [ ]:
N = 32
J = 1.0
T = 0.1
grid = Grid(N, J, T, "periodic")
@benchmark iterationWolff!(grid)

And the final result does make sense, at least on the visual aspect (the benchmark performed many iterations on the same grid, so we get a relatively valid sample after it is finished):

In [ ]:
show(grid)